## Preprocessing

In [20]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import save_model

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("Resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the unneeded ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(columns=['EIN', 'NAME'])


In [3]:
# Determine the number of unique values in each column.
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()
application_df[application_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
valCounts = application_df['APPLICATION_TYPE'].value_counts()
valCounts
## for cols with more than 10 unique values, pick a cutoff point to bin 'rare' categorical variables together in 'Other'

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T14        3
T25        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(valCounts[valCounts<500].index)
application_types_to_replace

# Replace in dataframe
for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C2380        1
C1820        1
C1570        1
C2150        1
C1283        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# You may find it helpful to look at CLASSIFICATION value counts >1
classCounts = application_df['CLASSIFICATION'].value_counts()
classValues = classCounts[classCounts>1]
classValues

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C2300       32
C7200       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C1260        3
C0           3
C2710        3
C1246        2
C1256        2
C1267        2
C1234        2
C3200        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = list(classCounts[classCounts<1000].index)
classifications_to_replace

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [9]:
# Convert categorical data to numeric with `pd.get_dummies`
dumdf = pd.get_dummies(application_df)
dumdf.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [10]:
# Split our preprocessed data into our features and target arrays
# define X (features) and y (target)
y = dumdf.IS_SUCCESSFUL.values
X = dumdf.drop(columns="IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [11]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [12]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model = tf.keras.models.Sequential()

# First hidden layer
nn_model.add(tf.keras.layers.Dense(units=16, activation="tanh", input_dim=43))

# Second hidden layer
nn_model.add(tf.keras.layers.Dense(units=16, activation="tanh"))

# Output layer
nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 16)                704       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 1)                 17        
                                                                 
Total params: 993
Trainable params: 993
Non-trainable params: 0
_________________________________________________________________


In [13]:
# Compile the model
nn_model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [14]:
# Train the model
fit_model = nn_model.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 3s 3ms/step - loss: 0.5853 - accuracy: 0.7095
Epoch 2/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5596 - accuracy: 0.7283
Epoch 3/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5550 - accuracy: 0.7280
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5525 - accuracy: 0.7290
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5511 - accuracy: 0.7312
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5497 - accuracy: 0.7305
Epoch 7/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5492 - accuracy: 0.7317
Epoch 8/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5483 - accuracy: 0.7316
Epoch 9/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5477 - accuracy: 0.7305
Epoch 10/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5478 - accuracy: 0.7322

In [15]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn_model.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5546 - accuracy: 0.7279 - 478ms/epoch - 2ms/step
Loss: 0.5546422600746155, Accuracy: 0.7279300093650818


In [16]:
# Let's try to lower the loss - 55% is high.
# Let's try adding another layer and changing our activation method.

# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
nn_model2 = tf.keras.models.Sequential()

# First hidden layer
nn_model2.add(tf.keras.layers.Dense(units=16, activation="relu", input_dim=43))

# Additional hidden layers
nn_model2.add(tf.keras.layers.Dense(units=16, activation="relu"))
nn_model2.add(tf.keras.layers.Dense(units=16, activation="tanh"))
# Output layer
nn_model2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn_model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 16)                704       
                                                                 
 dense_4 (Dense)             (None, 16)                272       
                                                                 
 dense_5 (Dense)             (None, 16)                272       
                                                                 
 dense_6 (Dense)             (None, 1)                 17        
                                                                 
Total params: 1,265
Trainable params: 1,265
Non-trainable params: 0
_________________________________________________________________


In [17]:
# Compile and train the new model
nn_model2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
fit_model = nn_model2.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5779 - accuracy: 0.7130
Epoch 2/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5573 - accuracy: 0.7271
Epoch 3/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5537 - accuracy: 0.7282
Epoch 4/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5523 - accuracy: 0.7296
Epoch 5/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5509 - accuracy: 0.7308
Epoch 6/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5502 - accuracy: 0.7305
Epoch 7/50
804/804 [==============================] - 2s 3ms/step - loss: 0.5493 - accuracy: 0.7316
Epoch 8/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5485 - accuracy: 0.7312
Epoch 9/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5481 - accuracy: 0.7325
Epoch 10/50
804/804 [==============================] - 2s 2ms/step - loss: 0.5480 - accuracy: 0.7322

In [18]:
# Evaluate the new model using the test data
model_loss2, model_accuracy2 = nn_model2.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss2}, Accuracy: {model_accuracy2}")

268/268 - 0s - loss: 0.5558 - accuracy: 0.7257 - 341ms/epoch - 1ms/step
Loss: 0.5558334589004517, Accuracy: 0.7257142663002014


In [ ]:
# Still 55% loss, with 74% accuracy. Let's try another approach...

# NOTE - The below function is commented out. It did not complete running all trials 
# before stopping due to performance issues. Of the approximately 70 trials run, 
# none performed better than the two above.

################################################################################

# # Create a method that creates a new Sequential model with hyperparameter options
# def create_model(hp):
#     nn_model = tf.keras.models.Sequential()

#     # Allow kerastuner to decide which activation function to use in hidden layers
#     activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
#     # Allow kerastuner to decide number of neurons in first layer
#     nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
#         min_value=4,
#         max_value=10,
#         step=2), activation=activation, input_dim=43))

#     # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
#     for i in range(hp.Int('num_layers', 1, 4)):
#         nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
#             min_value=4,
#             max_value=10,
#             step=2),
#             activation=activation))
    
#     nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#     # Compile the model
#     nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
#     return nn_model

# # Import the kerastuner library
# import keras_tuner as kt

# # Set the create_model function as keras tuner
# ## Running this one looks like tanh is the best activation method
# tuner = kt.Hyperband(
#     create_model,
#     objective="val_accuracy",
#     max_epochs=20,
#     hyperband_iterations=2)

# from kerastuner.tuners import RandomSearch
# tuner2 = RandomSearch(
#     create_model,
#     objective='val_accuracy',
#     max_trials=35,
#     executions_per_trial=5)

In [ ]:
# # Run the kerastuner random search for best hyperparameters
# ## Capped at 35 trials to save memory
# tuner2.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [ ]:
# # Run the kerastuner search for best hyperparameters
# tuner.search(X_train_scaled,y_train,epochs=20,validation_data=(X_test_scaled,y_test))

In [ ]:
# # Show a summary of the search
# tuner.results_summary()

# # Retrieve the best model
# best_model = tuner.get_best_models(num_models=1)[0]

# # Evaluate the best model
# loss, accuracy = best_model.evaluate(X_test, y_test)

## Loss 70%, Accuracy 46%! 
Running the function above (now commented out) is so much worse than our first two attempts, which we were never able to improve upon.

Let's save our second model, nn_model2, as it appears to be the best we can do with this data and our computational limitations.

In [22]:
# Export our model to HDF5 file
save_model(
    nn_model2,
    'nn_model2.h5'
)